[ja]: #
# Chapter 8. RCにおける閉ループ制御

[en]: #
# Chapter 8. Closed-loop control with Reservoir Computing

[ja]: #
この章では、RCを用いた閉ループ制御について学びます。
特にオフライン学習とオンライン学習の2つを学びます。

[en]: #
In this chapter, you will learn about **closed-loop control** in RC setups.
You will learn two major schemes of closed-loop control, i.e., offline and online.

[ja]: #
## 前書き

[en]: #
## Introduction

[ja]: #
ここまで主に開ループ系、すなわち出力が入力に影響を与えない系を用いたRCの手法を扱いました。
この章では**閉ループ系**、すなわち出力が入力に影響を与える設定を扱います。

このような閉ループ系は歴史的には制御工学の文脈で登場し、したがって機械やロボットの制御との大きな関連があります。
そして物理リザバー計算（PRC）でも、出力が環境やその系そのものに密接に相互作用している場合には不可避的に閉ループ系が登場し、その制御を考えなければなりません。
例えば柔らかい魚ロボットの身体と環境に内在する情報処理能力を活用し、その制御を物理系上で実現する研究<sup>[1]</sup> では閉ループ制御による制御信号の埋め込みによって魚の動きが実現されています。
ここではアクチュエータを介して魚の動きが生成され、環境とその身体形状に影響を及ぼし、リードアウト層に結合するセンサの値に影響を与える閉ループが構成されています。

[en]: #
So far, you have mainly learned RC techniques in open-loop systems, i.e., systems where the output does not affect the input.

This chapter introduces **closed-loop systems**, i.e., settings where the output affects the input.
Historically, such closed-loop systems have appeared in the context of control theory and are therefore closely related to the control of machines and robots.
Even in physical reservoir computing (PRC), closed-loop systems inevitably appear when the output closely interacts with the environment or the system itself.
For example, a soft robotic fish<sup>[1]</sup> utilizes the information processing capabilities intrinsic to the body and environment to realize its behavior.
In this case, the fish's movements are generated via actuators, which affect the environment and body shape, forming a closed loop that influences the sensor values connected to the readout layer.

[ja]: #
### 定式化

[en]: #
### Formulation

[ja]: #
PRC以前にも、数値計算の形でESNを用いた閉ループ制御はこれまで多くの研究で実証されてきました。
ここではこれまで扱われてきた以下のESNによる典型的な1入力1出力の開ループ系との対比で考えましょう。

[en]: #
Even before PRC, many numerical studies demonstrated closed-loop control using ESNs.
Let's consider the typical one-input, one-output open-loop system using ESNs as follows:

[END]: #

$$
\begin{align*}
x[k+1] &= \tanh\left(\rho W^\mathrm{rec} x[k] + W^\mathrm{in} u[k+1]\right)\\
y[k] &= W^\mathrm{out}[1 ; x[k]]
,\end{align*}
$$

[ja]: #
ここで $x[k]\in \mathbb{R}^N$ はESNの状態、$W^\mathrm{in} \in \mathbb{R}^{N\times 1}, W^\mathrm{rec} \in \mathbb{R}^{N\times N}, W^\mathrm{out} \in \mathbb{R}^{1\times (N+1)}$ は結合行列になります。
特に $W^\mathrm{out}$ は目標時系列 $d[k]$ を用いて $y[k]\approx d[k]$ となるように学習されます。
閉ループ制御では以下のような $W^\mathrm{feed} \in \mathbb{R}^{N\times 1}$ を加え出力 $y[k]$ がもう一度入力に作用します。

[en]: #
where $x[k]\in \mathbb{R}^N$ is the state of the ESN, and $W^\mathrm{in} \in \mathbb{R}^{N\times 1}, W^\mathrm{rec} \in \mathbb{R}^{N\times N}, W^\mathrm{out} \in \mathbb{R}^{1\times (N+1)}$ are the connection matrices.
In particular, $W^\mathrm{out}$ is trained to approximate the target time series $d[k]$ such that $y[k]\approx d[k]$.
In closed-loop control, we add $W^\mathrm{feed} \in \mathbb{R}^{N\times 1}$, and the output $y[k]$ is fed back as the input:

[END]: #

$$
\begin{align*}
x[k+1] &= \tanh\left(\rho W^\mathrm{rec} x[k] + W^\mathrm{in} u[k+1]+ W^\mathrm{feed} y[k] \right)\\
y[k] &= W^\mathrm{out}[1 ; x[k]]
.\end{align*}
$$

[ja]: #
またタスクが入力時系列の置き換え、すなわち $d[k] = u[k+1]$ の場合は、$W^\mathrm{in}$ を使用せずに以下のようになります。

[en]: #
When the task is to replace the input time series, i.e., $d[k] = u[k+1]$, you can omit $W^\mathrm{in}$ as follows:

[END]: #
$$
\begin{align*}
x[k+1] &= \tanh\left(\rho W^\mathrm{rec} x[k] + W^\mathrm{feed} y[k] \right)
.\end{align*}
$$

[ja]: #
ここで開ループ系と式はほとんど同じものの $W^\mathrm{out}$ が系全体の性質を変更する点に注意ください。
特にリードアウト層が線形な今回のケースでは、実質的にESNの内部結合が変化します（すなわち $\rho W^\mathrm{rec} + W^\mathrm{feed} W^\mathrm{out}$ ）。
実際Full-FORCE<sup>[2]</sup> や、後の章で紹介される Innate training <sup>[3]</sup>と呼ばれる手法では、このような内部結合と閉ループの等価性を活用し様々な制御のESN上での埋め込みを実証しています。

[en]: #
Note that although the equation is almost the same as the open-loop system, $W^\mathrm{out}$ changes the properties of the entire system.
Specifically, when the readout layer is linear as in this case, the internal coupling of the ESN is effectively altered after tuning the readout layer (i.e., $\rho W^\mathrm{rec} + W^\mathrm{feed} W^\mathrm{out}$).
Methods such as full-FORCE<sup>[2]</sup> and innate training<sup>[3]</sup>, which will be introduced in a later chapter, demonstrated various control tasks on ESNs by utilizing the equivalence between such internal coupling and closed loops.

[ja]: #
### オフライン学習とオンライン学習

[en]: #
### Offline and online learning

[ja]: #
RCにおける閉ループ制御、すなわち $W^\mathrm{out}$ の学習方法は**オフライン学習**と**オンライン学習**に大別されます。
オフライン学習はほとんど開ループ系の学習と同じで、線型回帰ないしはリッジ回帰によって達成されます。
その際、正解データである $d(t)$ が直接ESNに投射され $x(t)$ の時系列を $W^\mathrm{out}$ の学習に用いた後に評価フェーズで閉ループに切り替えられます。
このような学習方法は **教師強制 (teacher forcing)** と呼ばれます。
H. JaegerらによるRC最初の論文では、
教師強制でサンプルされたデータを基に、ESNに接続した線形の閉ループ結合を学習し、 **カオス時系列データ** の予測に、他のニューラルネットワークを用いた手法と比してより高い精度で成功しています<sup>[4]</sup>。

一方でオンライン学習は、逐次パラメータ $W^\mathrm{out}$ を更新する学習方式を指します。
このオンライン学習の手法の中で特にRCにおいて成功を収めたのがD. Sussilloらによって提唱されたFORCE（First-Order Reduced and Controlled Error）学習<sup>[5]</sup>です。
FORCE 学習は、[再帰最小二乗（Recursive Least Squares; RLS）フィルター](https://en.wikipedia.org/wiki/Recursive_least_squares_filter) と呼ばれる適応フィルター（適応制御に登場する手法）のアルゴリズムを利用して線形閉ループを調整します。
RLSフィルターを使用したオンライン学習は先述のH. Jaegerらによる論文で既に提案され実証されていますが、FORCE 学習の特異性はその使用されるネットワーク条件にあります。
つまり、FORCE 学習では初期設定として **カオス ESN** が使用されます。
これまで扱ったとおりESPが成り立たないカオスESNをリザバーとして使用するのは一見奇妙に見えるかもしれませんが、FORCE学習はシステムのカオス性をうまく活用し、カオス時系列を含む様々な目的のダイナミクスを設計できます。

[en]: #
Closed-loop control in RC, i.e., the learning method of $W^\mathrm{out}$, can be divided into **offline** and **online** learning.
Offline learning is almost the same as open-loop learning during the training phase and is achieved by linear regression or ridge regression.
The target time-series data $d(t)$ is directly projected onto the ESN, and $W^\mathrm{out}$ is trained based on the sampled state dynamics $x(t)$.
After training, the system switches to closed-loop mode in the evaluation phase.
This scheme is called **teacher forcing**.
The effectiveness of closed-loop control using ESNs was initially demonstrated by H. Jaeger et al. (2004)<sup>[4]</sup>, outperforming existing neural network-based methods in **chaotic time-series data prediction** tasks.

On the other hand, online learning refers to a learning method that sequentially updates the parameter $W^\mathrm{out}$.
FORCE (First-Order Reduced and Controlled Error) learning<sup>[5]</sup>, proposed by D. Sussillo et al., is one of the most successful online learning methods in RC.
FORCE learning adjusts the linear closed loop using an adaptive filter algorithm called a [recursive least squares (RLS) filter](https://en.wikipedia.org/wiki/Recursive_least_squares_filter).
An online scheme using an RLS filter had already been proposed and demonstrated by H. Jaeger et al.<sup>[4]</sup>, but the uniqueness of FORCE learning lies in the network conditions it uses.
Specifically, FORCE learning uses a **chaotic ESN** as the initial setting.
It might seem unusual to use a chaotic ESN as the reservoir since the ESP does not hold.
Surprisingly, FORCE learning leverages the system's chaoticity to design various dynamics, including complicated chaotic time series.

[ja]: #
## 演習問題と実演

[en]: #
## Exercises and demonstrations

[ja]: #
前回と同様、各種ライブラリおよび実装済みの関数の`import`を行うために次のセルを実行してください。なお内部実装を再確認するには、`import inspect`以下の行をコメントアウトするか`...?? / ??...`を使用してください。

[en]: #
Please execute the following cell to import the classes that we implemented previously, as well as the basic libraries.
You can check the internal implementations either by uncommenting the lines after `import inspect` or by using `...?? / ??...`.

In [ ]:
import sys

import numpy as np

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/[[PROJECT_NAME]]/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/[[PROJECT_NAME]]/
    else:
        pass
        %cd /content/
        !git clone --branch [[BRANCH_NAME]] https://github.com/rc-bootcamp/[[PROJECT_NAME]].git
        %cd /content/[[PROJECT_NAME]]/
else:
    sys.path.append(".")

from utils.reservoir import ESN, Linear, RidgeReadout
from utils.tester import load_from_chapter_name
from utils.tqdm import trange
from utils.viewer import (
    construct_delayed_coord,
    show_3d_coord,
    show_delayed_coord,
    show_record,
    show_return_map,
    show_trajectory,
)

test_func, show_solution = load_from_chapter_name("08_closed_loop_control")

# Uncomment it to see the implementations.
# import inspect
# print(inspect.getsource(Linear))
# print(inspect.getsource(ReReadout))
# print(inspect.getsource(ESN))

# Or just use ??.../...?? (uncomment the following lines).
# Linear??
# RidgeReadout??
# ESN??

[ja]: #
### 1. 目標時系列の生成

[en]: #
### 1. Generating target time-series data

[ja]: #
まず目標となる時系列を生成しましょう。
今回はカオス時系列を用意し、閉ループ系を用いて最終的にESN上で目標時系列を自律的に生成させましょう。
このようなタスクは「埋め込み」と呼ばれます。

[en]: #
Let's generate the target time series.
In this section, we'll prepare a chaotic time series and use an ESN to autonomously generate it in the closed loop.
This type of task is called "embedding."

[ja]: #
#### 目標時系列 #1（ローレンツ系）

[en]: #
#### Target time-series dataset #1 (Lorenz system)

[ja]: #
まずは前の章で実装したローレンツ系をルンゲ・クッタ法（RK4）で生成しましょう。

[en]: #
Let's generate the chaotic time-series data of the Lorenz system implemented in the previous chapter using the Runge–Kutta method (RK4).

In [ ]:
from utils.chaos import lorenz_func, runge_kutta

# Uncomment it to see the implementations.
# runge_kutta??
# lorenz_func??

dt = 0.01
time_steps = 20000
z0 = np.array([1.0, 1.0, 1.0])
lorenz_params = dict(a=10.0, b=28.0, c=8.0 / 3.0)
lorenz_func_rk4 = runge_kutta(dt, lorenz_func, **lorenz_params)

ds_lz = np.zeros((time_steps + 1, 3))
ds_lz[0] = z0
for idx in trange(time_steps):
    ds_lz[idx + 1] = lorenz_func_rk4(ds_lz[idx])

[ja]: #
3次元座標上でローレンツ系の時系列を可視化しましょう。

[en]: #
Let's visualize the time series of the Lorenz system in 3D coordinates.

In [ ]:
fig = show_3d_coord(ds_lz)
fig.show()

[ja]: #
各 $x(t), y(t), z(t)$ に対して時間遅れ座標 $(x(t), x(t + 10 \Delta t))$ を表示し、同様に特徴的なdouble wingが見られるか確認しましょう。

[en]: #
The following cell displays the time-delay coordinates $(x(t), x(t + 10 \Delta t))$ for each of $x(t), y(t), z(t)$.
Check if you can see the characteristic "double wing" attractor.

In [ ]:
for idx, name in enumerate("xyz"):
    tau = 10
    fig = show_delayed_coord(ds_lz[:, idx], tau=tau)
    fig[0].set_title(r"$\left({{{}}}(t), {{{}}}(t+{{{}}}\Delta t)\right)$".format(name, name, tau))

[ja]: #
この図は、部分的な観測からでも元の力学系に関する情報を再構築できる可能性を示唆しています ([Takensの埋め込み定理](https://en.wikipedia.org/wiki/Takens%27s_theorem)を参照)。

[en]: #
This plot suggests that we can reconstruct information about the original dynamical system even from partial observations (see [Takens' embedding theorem](https://en.wikipedia.org/wiki/Takens%27s_theorem)).

[ja]: #
#### 目標時系列 #2（マッキー・グラス方程式）

[en]: #
#### Target time-series dataset #2 (Mackey-glass equation)

[ja]: #
H. Jaegerらによる最初の論文 <sup>[4]</sup>では、白血球細胞の生成過程をモデル化したマッキー・グラス（Mackey-Glass; MG）方程式<sup>[6]</sup>と呼ばれる**遅延微分方程式**を使用しています。
遅延微分方程式は、現時点の状態のみならず過去の状態にもその導関数が依存するような微分方程式を指します。
実際MG方程式は以下の式のような遅延項 $x(t-\tau)$ を含んだ形で表現されます。

[en]: #
H. Jaeger et al.<sup>[4]</sup> used a delay differential equation called the Mackey-Glass (MG) equation<sup>[6]</sup>, which models the process of white blood cell generation.
A delay differential equation is a differential equation whose derivative depends not only on the current state but also on past states.
The MG equation is expressed in a form that includes a delay term $x(t-\tau)$, as shown in the following formula:

[END]: #

$$
\begin{align*}\\
\frac{dx}{dt}(t) = \frac{\beta x(t-\tau)}{1 + x(t-\tau)^{n}} - \gamma x(t)
,\end{align*}
$$

[ja]: #
ここで $\beta, \gamma, n, \tau$ はパラメータを表します。
MG方程式は様々な領域でカオス的挙動が報告されており、実際H. Jaegerらは $\beta=0.2, \gamma=0.1, n=10, \tau=17$ を採用しています。
このMG方程式によって生成される時系列そのものは1次元であるものの、$t=t_0$ における値 $x(t_0)$ のみならず $t \in [t_0 -\tau, t_0]$ の情報がその後の求積（時間発展）には必要になり、この意味でMG方程式は無限次元力学系です。
またより一般に遅延微分方程式は連続関数の空間を相空間とする無限次元力学系とみなせます。
しかしながら、計算機上では無限の状態を保持できず、したがってMG方程式の計算には工夫が必要です。

[en]: #
where $\beta, \gamma, n, \tau$ are the system's parameters.
The parameter set $\beta=0.2, \gamma=0.1, n=10, \tau=17$ is widely used to produce chaotic time-series data.
Though the time series generated by the MG equation is one-dimensional, all information in the interval $t \in [t_0 -\tau, t_0]$ is required for subsequent integration after $t=t_0$, meaning that the MG equation is an infinite-dimensional dynamical system.
More generally, delay differential equations can be regarded as infinite-dimensional dynamical systems whose phase space is the space of continuous functions.
However, infinite states cannot be held on a computer since it requires infinite memory.

[ja]: #
このような状況でよく用いられるのが履歴を状態とみなすテクニックです。
いま時間幅 $\Delta T$ を用いて$ [t_0-\tau, t_0]$ を $N (\in \mathbb{N})$ 分割します（$ \Delta T = \tau / (N - 1) $）。
このとき新たに $g(t) = [x(t-\tau)\quad  x(t-\tau + \Delta T) \quad \cdots \quad x(t-\Delta T) \quad x(t)]^\top \in \mathbb{R}^{N} $ を状態として定義します。
すると上記の式は以下のとおり書けます。

[en]: #
In this situation, a useful technique is to treat the history as the state.
Suppose you divide the time range $[t_0-\tau, t_0]$ into $N (\in \mathbb{N})$ intervals using a constant time width $\Delta T$ ($\Delta T = \tau / (N - 1)$).
You can define $g(t) = [x(t-\tau)\quad  x(t-\tau + \Delta T) \quad \cdots \quad x(t-\Delta T) \quad x(t)]^\top \in \mathbb{R}^{N}$ as the state.
Then the above equation can be rewritten as follows:

[END]: #

$$
\begin{align*}\\
\frac{dx}{dt}(t) &= \frac{dg_{N-1}}{dt}(t) \\
&= \frac{\beta g_{0}(t)}{1 + g_{0}(t)^{n}} - \gamma \cdot g_{N-1}(t)
,\end{align*}
$$

[ja]: #
この式はRK4等で積分すると $x(t+\Delta T)$ が求まります。
あとは $g_{k}(t + \Delta T) = g_{k+1} (t)$ によってずらせば解がえられます。
この処理を実装してみましょう。

[en]: #
by which you can get $x(t+\Delta T)$.
Then you can shift the state by $g_{k}(t + \Delta T) = g_{k+1} (t)$ to get the solution.
Let's implement this procedure.

Q1.1.

[ja]: #
上記の説明を基に、MG方程式による時系列を生成する関数 `mackey_glass_func` と `sample_mg_dynamics` を完成させよ。

[en]: #
Based on the above explanation, implement the functions `mackey_glass_func` and `sample_mg_dynamics` to generate the time series of the MG equation.

[END]: #

- `mackey_glass_func`
  - Argument(s):
    - `gn`: `np.ndarray | float`
    - `g0`: `np.ndarray | float`
  - Return(s):
    - `gn_dot`: `np.ndarray | float`

- `sample_mg_dynamics`
  - Argument(s):
    - `time_steps`: `int`
    - `tau`: `np.ndarray | float`
    - `num_split`: `int`
  - Return(s):
    - `ts`: `np.ndarray`
      - shape: `(time_steps,)`
    - `out`: `np.ndarray`
      - shape: `(time_steps, ...)`

In [ ]:
def mackey_glass_func(gn, g0, beta: float = 0.2, gamma: float = 0.1, n: float = 10.0):
    gn_dot = beta * g0 / (1 + g0**n) - gamma * gn  # RIGHT
    return gn_dot


def sample_mg_dynamics(
    time_steps: int,
    tau: float,
    num_split: int,
    values_before_zero=lambda t: 0.5,
    display=True,
    **kwargs,
):
    t_pre = np.linspace(-tau, 0, num_split)
    dt = t_pre[1] - t_pre[0]
    gs = [values_before_zero(t) for t in t_pre]

    out = None
    ts = np.arange(time_steps) * dt
    for idx in trange(time_steps, display=display):
        mg_func_rk4 = runge_kutta(dt, mackey_glass_func, g0=gs[0], **kwargs)  # RIGHT Use `runge_kutta`.
        # BEGIN Update `gs` using `mg_func_rk4`.
        gn = mg_func_rk4(gs[-1])
        gs = gs[1:] + [gn]
        # END
        if out is None:
            out = np.zeros((time_steps, *gs[-1].shape))
        out[idx] = gs[-1]
    return ts, out


test_func(sample_mg_dynamics, "01_01", multiple_output=True)
# show_solution("01_01", "sample_mg_dynamics")  # Uncomment it to see the solution.

[ja]: #
実装した関数でMG方程式のダイナミクスを描画しましょう。
以下のセルは時間遅れ座標 $(x(t), x(t-\tau))$ を描画します。

[en]: #
Let's plot the dynamics of the MG equation using the implemented function.
The following cell displays the time-delay coordinates $(x(t), x(t-\tau))$.

In [ ]:
tau = 17.0
dt = 1.0
num_split = int(tau // dt + 1)
time_steps = 20000

ts, ds_mg = sample_mg_dynamics(time_steps, tau, num_split, lambda t: 0.5)

fig = show_delayed_coord(ds_mg, tau=num_split - 1)

[ja]: #
3次元の時間遅れ座標 $(x(t), x(t-\tau), x(t-2\tau))$ だとどうなるでしょうか？

[en]: #
What happens if you use three-dimensional time-delay coordinates $(x(t), x(t-\tau), x(t-2\tau))$?

In [ ]:
data = construct_delayed_coord(ds_mg[10000:], num_split - 1, 3)
fig = show_3d_coord(data)
fig.show()

Q1.3. (Advanced)

[ja]: #
- MG方程式の各種パラメータに関して分岐図を書け。
- 時間幅が可変な求積法も存在する。このような状況で今回のようなテクニックは使用できない。遅延微分方程式の求積方法に関して調査し、Pythonで実装せよ。

[en]: #
- Draw a bifurcation diagram for the MG equation parameters (e.g., $\beta, \gamma, n$).
- There are also quadrature methods with adaptive time steps, where the technique presented here cannot be used. Survey quadrature methods for delay differential equations and implement them in Python.

[ja]: #
### 2. オフライン学習

[en]: #
### 2. Offline learning

[ja]: #
目標となる時系列データが用意できたのでオフライン学習を実装をしましょう。
まずは閉ループと開ループを切り替え、ESNの状態の時系列をサンプルする関数を実装します。

[en]: #
Now that you have prepared the target time series data, let's implement offline learning.
First, you will design a function that switches between open-loop and closed-loop modes corresponding to the training and evaluation phases, respectively.

Q2.1.

[ja]: #
以下の空欄を埋め、開ループと閉ループと切り替えESNの時系列を記録する `emulate_offline` を完成させよ。
ただし $k\in[t_0, t_1)$ のときは開ループ系 $x[k+1]=f(x[k], d[k])$、それ以外のときは閉ループ系 $x[k+1]=f(x[k],W^\mathrm{out} x[k])$ を使用する。

[en]: #
Implement `emulate_offline` that records the dynamics of the ESN.
Note that the system switches to open-loop mode $x[k+1]=f(x[k], d[k])$ when $k\in[t_\mathrm{b}, t_\mathrm{e})$ , and otherwise to the closed-loop system $x[k+1]=f(x[k],W^\mathrm{out} x[k])$.

[END]: #
- `emulate_offline`:
  - Argument(s):
    - `time_steps`: `int`
      - $T$
    - `x0`: `np.ndarray`
      - $x[0]$
    - `net`: `ESN`
    - `w_feed`: `Linear | RidgeReadout`
      - $W^\mathrm{feed}$
    - `w_out`: `Linear | RidgeReadout`
      - $W^\mathrm{out}$
    - `ds`: `np.ndarray`
      - $[d[0], d[1],~\ldots]$
    - `open_range`: `tuple(int, int)`
      - $[t_\mathrm{b}, t_\mathrm{e})$
  - Return(s):
    - `record`: `dict`
      - `'t'`: `np.ndarray`
        - $[0, 1,~\ldots,~T-1, T]$
      - `'x'`: `np.ndarray`
        - $[x[0], x[1],~\ldots,~x[T-1], x[T]]$
      - `'y'`: `np.ndarray`
        - $[y[0], y[1],~\ldots,~y[T-1]]$

[tips]: #
[ja]: #
- `ds` が `None` かつ $k \in [t_\mathrm{b}, t_\mathrm{e})$ の場合、出力 `y` をシステムにフィードバックする。
- それ以外の場合は、`ds[idx]` を使用する。

[en]: #
- When `ds` is `None` and $t \in [t_\mathrm{b}, t_\mathrm{e})$, the system will feed the output `y` back into the system.
- Otherwise, it will use `ds[idx]`.

[/tips]: #

In [ ]:
def emulate_offline(
    time_steps,
    x0,
    net,
    w_feed,
    w_out,
    ds=None,
    open_range=None,
    label=None,
    display=True,
):
    open_range = open_range if open_range is not None else [0, 0]
    record = {}
    record["t"] = np.arange(0, time_steps + 1)
    record["x"] = np.zeros((time_steps + 1, *x0.shape))
    record["y"] = np.zeros((time_steps, *w_out(x0).shape))
    record["open_range"] = open_range
    x = x0
    record["x"][0] = x0
    pbar = trange(time_steps, display=display)
    for idx in pbar:
        if label is not None:
            pbar.set_description(label)
        if (ds is not None) and (idx < len(ds)) and (open_range[0] <= idx < open_range[1]):
            y = ds[idx]  # RIGHT Training phase (i.e., teacher forcing by `ds`)
        else:
            y = w_out(x)  # RIGHT Evaluation phase (i.e., autonomous closed-loop mode by `w_out`)
        x = net(x, w_feed(y))  # RIGHT Implement x[k + 1] = f(x[k], y[k])
        record["x"][idx + 1] = x
        record["y"][idx] = y
    return record


def solution(*args, **kwargs):
    record = emulate_offline(*args, **kwargs)
    return record["x"], record["y"]


test_func(solution, "02_01", multiple_output=True)
# show_solution("02_01", "emulate_offline")  # Uncomment it to see the solution.

[ja]: #
実装した `emulate_offline` を用いて時系列を収集し、そのデータを基に $W^\mathrm{out}$ を学習しましょう。
以下の設問はそのような処理を実現する関数 `run_train_and_eval` を実装します。

[en]: #
Let's collect time-series data using the implemented `emulate_offline` and train $W^\mathrm{out}$ based on that data.
The following exercise implements the function `run_train_and_eval` to achieve this procedure.

Q2.2.

[ja]: #
以下の空欄を埋め、開ループ系における教師強制によるサンプリングと、閉ループ系における評価を実施する `run_train_and_eval` を実装せよ。

[en]: #
Implement `run_train_and_eval` to perform the learning procedures of the training and evaluation phases.

[END]: #
- `run_train_and_eval`:
  - Argument(s):
    - `t_washout, t_train, t_eval`: `int`
      - $T_\mathrm{washout}, T_\mathrm{train}, T_\mathrm{eval}$
    - `x0`: `np.ndarray`
      - $x[0]$
    - `net`: `ESN`
    - `w_feed`: `Linear | RidgeReadout`
      - $W^\mathrm{feed}$
    - `w_out`: `Linear | RidgeReadout`
      - $W^\mathrm{out}$
    - `ds`: `np.ndarray`
      - $[d[0], d[1],~\ldots]$

[ja]: #
  - Return(s):
    - `record_t`: `dict`
      - 学習フェーズにおける時系列
    - `record_e`: `dict`
      - 評価フェーズにおける時系列

[en]: #
  - Return(s):
    - `record_t`: `dict`
      - training data
    - `record_e`: `dict`
      - evaluation data

In [ ]:
def run_train_and_eval(t_washout, t_train, t_eval, x0, net, w_feed, w_out, ds, display=True):
    # Training phase (open loop)
    record_t = emulate_offline(
        t_washout + t_train,
        x0,
        net,
        w_feed,
        w_out,
        ds,
        open_range=[0, t_washout + t_train],
        label="Train",
        display=display,
    )

    # Run ridge regression and update of the weight
    x_train = record_t["x"][t_washout : t_washout + t_train]  # RIGHT t \in [t_washout, t_washout + t_train)
    y_train = record_t["y"][t_washout : t_washout + t_train]  # RIGHT t \in [t_washout, t_washout + t_train)
    y_eval = ds[t_washout + t_train : t_washout + t_train + t_eval]
    w_out.train(x_train, y_train)

    # Evaluation phase (closed loop)
    x1 = np.array(record_t["x"][-1])
    record_e = emulate_offline(t_eval, x1, net, w_feed, w_out, label="Eval", display=display)
    record_e["d"] = y_eval
    return record_t, record_e


def solution(*args, **kwargs):
    # DO NOT CHANGE HERE.
    _record_t, record_e = run_train_and_eval(*args, **kwargs)
    return record_e["x"], record_e["y"]


test_func(solution, "02_02", multiple_output=True)
# show_solution("02_02")  # Uncomment it to see the solution.

[ja]: #
#### 非カオスESNを用いたデモンストレーション

[en]: #
#### Demonstration with non-chaotic ESN

[ja]: #
ここからは実際に生成した目標時系列を非カオスESNを用いて埋め込みます。
まずはローレンツ系の予測をスペクトル半径 $\rho=0.1$ のESNを用いて行います。
このときリードアウト層の出力次元が3である点に注意ください（`out_dim = 3`）。

[en]: #
Now you will embed the generated target time series using a non-chaotic ESN.
Let's predict the Lorenz system using an ESN with a spectral radius $\rho=0.1$.
Note that the output dimension of the readout layer is 3 (`out_dim = 3`).

In [ ]:
seed = 1234
rnd = np.random.default_rng(seed)

net_dim = 1000
out_dim = 3
t_washout, t_train, t_eval = 2000, 8000, 5000

net = ESN(net_dim, sr=0.1, p=0.01, a=None, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=1.0, bias=0.2, rnd=rnd)
w_out = RidgeReadout(
    net_dim, out_dim, lmbd=0, rnd=rnd
)  # Set positive value to alpha (e.g., alpha=1e-6) if output is unstable.
x0 = rnd.uniform(-1, 1, net_dim)

ds = (ds_lz * 0.01)[:, :out_dim]
rec_lz_t, rec_lz_e = run_train_and_eval(t_washout, t_train, t_eval, x0, net, w_feed, w_out, ds)

fig_t = show_record(rec_lz_t, ["x", "y"])
fig_t[0].set_title("Training")
fig_e = show_record(rec_lz_e, ["x", "y"])
fig_e[0].set_title("Evaluation")

None

[ja]: #
ローレンツ系の埋め込みが成功したかどうかを定性的に評価するため、3次元座標のでアトラクタの形と、リターンマップを確認します。

[en]: #
To qualitatively evaluate whether the embedding of the Lorenz system was successful, the following cell displays the shape of the attractor in 3D coordinates and the return map.

In [ ]:
fig = show_return_map(output=rec_lz_e["y"][:, 2], desired=rec_lz_e["d"][:, 2])

fig = show_3d_coord(output=rec_lz_e["y"], desired=rec_lz_e["d"])
fig.show()

[ja]: #
今度はMG方程式による時間遅れ時系列をスペクトル半径 $\rho=0.9$ のESNに埋め込みましょう。

[en]: #
Next, let's embed the time-delayed time series from the MG equation into a non-chaotic ESN with a spectral radius $\rho=0.9$.

In [ ]:
seed = 1234
rnd = np.random.default_rng(seed)

net_dim = 1000
out_dim = 1
t_washout, t_train, t_eval = 2000, 8000, 5000

net = ESN(net_dim, sr=0.9, p=0.01, a=None, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=0.5, bias=0.5, rnd=rnd)
w_out = RidgeReadout(
    net_dim, out_dim, lmbd=0, rnd=rnd
)  # Set positive value to alpha (e.g., alpha=1e-6) if output is unstable.
x0 = rnd.uniform(-1, 1, net_dim)

ds = (ds_mg - 1.0)[:, None]
rec_mg_t, rec_mg_e = run_train_and_eval(t_washout, t_train, t_eval, x0, net, w_feed, w_out, ds)

fig_t = show_record(rec_mg_t, ["x", "y"])
fig_t[0].set_title("Training")
fig_e = show_record(rec_mg_e, ["x", "y"])
fig_e[0].set_title("Evaluation")

None

[ja]: #
遅延遅れ座標系を構成し、目標軌道と合わせて3次元座標系上に表示してみます。

[en]: #
The following cell constructs a delay coordinate system and displays it in 3D coordinates along with the desired trajectory.

In [ ]:
data1 = construct_delayed_coord(rec_mg_e["y"], 17, 3)
data2 = construct_delayed_coord(rec_mg_e["d"], 17, 3)
fig = show_3d_coord(output=data1, desired=data2)
fig.show()
# fig = show_return_map(output=rec_mg_e['y'][:, 0], desired=rec_mg_e['d'][:, 0])

[ja]: #
ローレンツ系、MG方程式いずれのケースも、ESNが自律的に生成できる様子が確認できると思います。

[en]: #
You can see that the ESN autonomously generates both the Lorenz system and the MG equation simply by tuning the linear feedback loop.

Q2.2. (Advanced)

[ja]: #
- $\rho$ や他のパラメータを変え学習の安定性を調査せよ。
- カオス時系列は初期値鋭敏性を有するため、仮に高い精度でアトラクタの構成に成功しても目標時系列との差分が指数的に拡大してしまい MSE等の指標が使えない。
埋め込みの可否を定量的に評価する指標を考え実装せよ。
- 1次元のみの時系列からのローレンツ系の埋め込みを試し（`out_dim = 1`に変更せよ）、学習がうまく行くパラメータのセットを探せ。

[en]: #
- Investigate the learning stability by changing $\rho$ and other parameters.
- Due to the initial sensitivity of chaotic attractors, error-based metrics such as MSE are ineffective since the difference from the target time series grows exponentially even when the embedding is successful. Design and implement a metric to quantitatively evaluate the success of embedding.
- Try embedding the Lorenz system using only a one-dimensional time series (change to `out_dim = 1`) and find a set of parameters that work well.

[ja]: #
#### カオスESNを用いたデモンストレーション

[en]: #
#### Demonstration with chaotic ESN

[ja]: #
ここまでは非カオスESN（$\rho<1$）を用いてオフライン学習を行いました。
カオスESNの場合はどうなるか検証しましょう。
以下のセルは先ほどとほぼ同じですが $\rho$ のみが $1.5$ に変更されています。

[en]: #
So far, you have examined the behavior of offline learning using a non-chaotic ESN ($\rho<1$).
Let's verify what happens with a chaotic ESN.
The following cell is almost the same as before, except that $\rho$ has been changed to $1.5$.

In [ ]:
seed = 1234
rnd = np.random.default_rng(seed)

net_dim = 1000
out_dim = 3
t_washout, t_train, t_eval = 2000, 8000, 5000

net = ESN(net_dim, sr=1.5, p=0.01, a=None, rnd=rnd)  # Spectral radius 1.5
w_feed = Linear(out_dim, net_dim, bound=1.0, bias=0.2, rnd=rnd)
w_out = RidgeReadout(
    net_dim, out_dim, lmbd=0, rnd=rnd
)  # Set positive value to alpha (e.g., alpha=1e-6) if output is unstable.
x0 = rnd.uniform(-1, 1, net_dim)

ds = (ds_lz * 0.01)[:, :out_dim]
rec_lz_t, rec_lz_e = run_train_and_eval(t_washout, t_train, t_eval, x0, net, w_feed, w_out, ds)

fig_t = show_record(rec_lz_t, ["x", "y"])
fig_t[0].set_title("Training")
fig_e = show_record(rec_lz_e, ["x", "y"])
fig_e[0].set_title("Evaluation")

[ja]: #
3次元座標上でのアトラクタの形を確認しましょう。

[en]: #
Let's check the shape of the attractor in 3D coordinates.

In [ ]:
fig = show_3d_coord(output=rec_lz_e["y"], desired=rec_lz_e["d"])
fig.show()

[ja]: #
ご覧のとおり、$\rho < 1$ のケースよりもうまくいっていない様子が見て取れます（`seed`や他のパラメータを変えて試して見てください）。
一般にオフライン学習の場合、カオスESNを用いた学習が一般に困難である場合が多いです。

[en]: #
This does not work as well as the case where $\rho < 1$.
In general, it is known that offline learning with chaotic ESNs is challenging.

[ja]: #
### 3. オンライン学習

[en]: #
### 3. Online learning

[ja]: #
今度はオンライン学習を実装してみましょう。
特にここではFORCE学習ならびにその背後で使用されるRLSアルゴリズムを実装します。

[en]: #
Next, let's implement online learning, specifically FORCE learning and the RLS algorithm.

[ja]: #
#### RLSアルゴリズム

[en]: #
#### RLS algorithm

[ja]: #
まずRLSアルゴリズムを実装します。
RLSアルゴリズムは非常に収束性が高い適応フィルタの一種として知られ以下の一連の式で表現されます。

[en]: #
First, you will implement the RLS algorithm.
The RLS algorithm is an adaptive filter known for its high convergence and is expressed by the following series of equations:

[END]: #

$$
\begin{align*}\\
k &= P x \\
g &= \frac{1}{1+x^\top k} \\
\Delta P &= g k k^\top \\
\Delta w^{\mathrm{out}} &= g (d - y) k^\top \\
P &\leftarrow P - \Delta P \\
w^\mathrm{out} &\leftarrow w^\mathrm{out} + \Delta w^\mathrm{out}
,\end{align*}
$$

[ja]: #
ここで、$P \in \mathbb{R}^{N\times N}$ は正定値行列、$y, d$ はそれぞれシステム出力と目的出力、$x$ はシステムの状態です。
RLSアルゴリズムは、$P(0)$ の初期値を $I/\lambda$ に設定すると、正則化項 $\lambda$ を使用したリッジ回帰と同等になります。
すなわち以下の式として表現されます。

[en]: #
where $P \in \mathbb{R}^{N\times N}$ is a positive definite matrix, $y$ and $d$ are the system output and target output, respectively, and $x$ is the system state.
By setting the initial value of $P(0)$ to $I/\lambda$, the RLS algorithm becomes equivalent to ridge regression with the regularization term $\lambda$, so that the following equation holds:

[END]: #

$$
\begin{align*}\\
P[T]&=\left(\sum_{k=1}^{T} {x}[k]{x}^{\top}[k] + \lambda I\right)^{-1} \\
&=\left(X^\top X + \lambda I\right)^{-1}
,\end{align*}
$$

[ja]: #
ここで、$X=[{x}^\top[1]; x^\top[1]; \cdots x^\top[T]]\in \mathbb{R}^{T\times N}$ は学習データです。
RLS アルゴリズムでは、[シャーマン・モリソンの公式 (ウッドベリー行列恒等式の特殊系) ](https://en.wikipedia.org/wiki/Woodbury_matrix_identity) を利用して、この逆行列の更新を効率的に計算します。
以下の式で $A=P^{-1}, B=x, C=x^\top, D=1$ を代入して上記の式の正しさを確認してください。

[en]: #
where $X=[{x}^\top[1]; x^\top[1]; \cdots x^\top[T]]\in \mathbb{R}^{T\times N}$ is the training data.
The RLS algorithm efficiently updates the inverse matrix using the Sherman–Morrison formula, which is a special case of the [Woodbury matrix identity](https://en.wikipedia.org/wiki/Woodbury_matrix_identity).
You can check the correctness of the above equation by substituting $A=P^{-1}, B=x, C=x^\top, D=1$ into the following formula:

[END]: #
$$
\begin{align*}
(A+BDC)^{-1} = A^{-1} - A^{-1}B(D^{-1}+CA^{-1}B)^{-1}CA^{-1}
.\end{align*}
$$

[ja]: #
制御工学になれ親しんだ方はこのRLSの更新式が、カルマンフィルタのそれとよく似ている点に気が付かれるでしょう。
実際RLSアルゴリズムもカルマンフィルタもどちらも推定アルゴリズムの一種であり、いくつかの前提条件によりカルマンフィルタの特殊系としてRLSアルゴリズムは位置づけられます。

[en]: #
Those familiar with control theory will notice that the update equation for RLS is very similar to that of the Kalman filter.
Both the RLS algorithm and the Kalman filter are estimation algorithms, and under certain assumptions, the RLS algorithm can be viewed as a special case of the Kalman filter.

[ja]: #
またFORCE学習は、その名前のとおり「1次の制御された誤差（First-Order Reduced and Controlled Error）」が制御されます。
RLSアルゴリズムによる更新前の誤差と更新後の誤差をそれぞれ $e^-[t], e^+[t]$としたとき、FORCE学習では常に $|e^+[t]|<|e^-[t]|$ に、また $e^+[t]/e^-[t]\to 1~(t \gg 1)$ になるように重みが制御されます。
その収束性に関わらず、このような条件を満たす学習アルゴリズムであればFORCE学習に当てはまります<sup>[5]</sup>。

[en]: #
Note that FORCE learning controls the "first-order reduced and controlled error," meaning that the errors before and after the RLS algorithm update, $e^-[t]$ and $e^+[t]$, are controlled to satisfy $|e^+[t]| < |e^-[t]|$ and $e^+[t]/e^-[t] \to 1~(t \gg 1)$.
Any learning algorithm that satisfies these conditions can be considered FORCE learning, regardless of its convergence speed<sup>[5]</sup>.

Q3.1.

[ja]: #
上記の説明を基に、FORCE学習ならびにRLSアルゴリズムによって重みを逐次更新する線形層のクラス `FORCEReadout` を実装せよ。

[en]: #
Based on the above explanation, implement a linear layer class `FORCEReadout` that iteratively updates the weights using the FORCE learning and RLS algorithms.

[END]: #

- `rls_update`
  - Argument(s)
    - `P`: `np.ndarray`
      - `shape`: `(n, n)`
      - `dtype`: `np.float64`
    - `x`: `np.ndarray`
      - `shape`: `(n,)`
      - `dtype`: `np.float64`
  - Return(s)
    - `g`: `np.ndarray`
      - `shape`: `()`
      - `dtype`: `np.float64`
    - `k`: `np.ndarray`
      - `shape`: `(..., 1, n)`
      - `dtype`: `np.float64`
    - `P_new`: `np.ndarray`
      - `shape`: `(..., n, n)`
      - `dtype`: `np.float64`

- `FORCEReadout.step`
  - Argument(s)
    - `x`: `np.ndarray`
    - `d`: `np.ndarray`
  - Return(s)
    - `dws`: `np.ndarray`

[ja]: #
  - Operation(s)
    - `rls_update`を用いた`ForceReadout.P`の更新
    - `FORCEReadout.weight` の更新

[en]: #
  - Operation(s)
    - Update `FORCEReadout.P` with `rls_update`.
    - Update `FORCEReadout.weight` with the obtained weight.

[tips]: #

- [`np.dot`](https://numpy.org/doc/stable/reference/generated/numpy.dot.html)
- [`np.outer`](https://numpy.org/doc/stable/reference/generated/numpy.outer.html)

[/tips]: #

In [ ]:
def rls_update(P, x):
    k = np.dot(P, x)  # RIGHT
    g = 1 / (1 + x.dot(k))  # RIGHT
    dP = g * np.outer(k, k)  # RIGHT
    P_new = P - dP  # RIGHT
    return g, k, P_new


class FORCEReadout(Linear):
    def __init__(self, *args, lmbd=1.0, initialize_with_zero=True, **kwargs):
        super(FORCEReadout, self).__init__(*args, **kwargs)
        self.P = np.eye(self.input_dim, dtype=self.dtype) * (1 / lmbd)  # RIGHT Create initial value of P.
        if initialize_with_zero:
            self.weight[:] = 0
            self.bias[:] = 0

    def step(self, x, d):
        assert x.ndim == 1
        e = d - self(x)
        g, k, P_new = rls_update(self.P, x)  # RIGHT Use `rls_update`.
        dw = g * np.outer(e, k)  # RIGHT Calculate `dw`.
        self.P = P_new
        self.weight += dw
        return dw


def solution(dim_in, dim_out, x_train, y_train, x_eval):
    # DO NOT CHANGE HERE.
    readout = FORCEReadout(dim_in, dim_out)
    readout.step(x_train, y_train)
    return readout(x_eval)


test_func(solution, "03_01")
# show_solution("03_01", "rls_update")  # Uncomment it to see the solution.
# show_solution("03_01", "FORCEReadout")  # Uncomment it to see the solution.

[ja]: #
#### FORCE学習による重みの更新

[en]: #
#### Updating weight by FORCE learning

[ja]: #
次にFORCE学習によって重みを更新する関数を実装しましょう。

[en]: #
Next, let's implement a function that updates the weights using FORCE learning.

Q3.2.

[ja]: #
以下の空欄を埋め、FORCE学習によって $W^\mathrm{out}$ を時々刻々更新する `emulate_online` を完成させよ。
ただし $k\in[t_0, t_1) \land k \equiv 0~ (\mathrm{mod}~t_\mathrm{every}) $ のときに $W^\mathrm{out}$ を更新、それ以外のときは変更しない。

[en]: #
Implement `emulate_online`, which uses FORCE learning to update $W^\mathrm{out}$.
Note that $W^\mathrm{out}$ is updated when $k\in[t_\mathrm{b}, t_\mathrm{e}) \land k \equiv 0~ (\mathrm{mod}~t_\mathrm{every}) $, and remains fixed otherwise.

[END]: #
- `emulate_offline`:
  - Argument(s):
    - `time_steps`: `int`
      - $T$
    - `x0`: `np.ndarray`
      - $x[0]$
    - `net`: `ESN`
    - `w_feed`: `Linear | RidgeReadout`
      - $W^\mathrm{feed}$
    - `w_out`: `Linear | RidgeReadout`
      - $W^\mathrm{out}$
    - `ds`: `np.ndarray`
      - $[d[0], d[1], \ldots]$
    - `force_every`: `int`
      - $t_\mathrm{every}$
    - `train__range`: `tuple(int, int)`
      - $[t_\mathrm{b}, t_\mathrm{e})$
  - Return(s):
    - `record`: `dict`
      - `'t'`: `np.ndarray`
        - $[0, 1, \ldots, T-1, T]$
      - `'x'`: `np.ndarray`
        - $[x[0], x[1], \ldots, x[T-1], x[T]]$
      - `'y'`: `np.ndarray`
        - $[y[0], y[1], \ldots, y[T-1]]$
      - `'d'`: `np.ndarray`
        - $[d[0], d[1], \ldots, d[T-1]]$
      - `'w'`: `np.ndarray`
        - $[W^\mathrm{out}[0], W^\mathrm{out}[1], \ldots, W^\mathrm{out}[T-1], W^\mathrm{out}[T]]$

In [ ]:
def emulate_online(
    time_steps,
    x0,
    net,
    w_feed,
    w_out,
    ds=None,
    train_range=None,
    force_every=1,
    display=True,
):
    train_range = train_range if train_range is not None else [0, 0]
    record = {}
    record["t"] = np.arange(0, time_steps + 1)
    record["x"] = np.zeros((time_steps + 1, *x0.shape))
    record["y"] = np.zeros((time_steps, *w_out(x0).shape))
    record["d"] = np.zeros((time_steps, *w_out(x0).shape))
    record["w"] = np.zeros((time_steps + 1, *w_out.weight.shape))
    record["train_range"] = train_range

    x = x0
    record["x"][0] = x0
    record["w"][0] = w_out.weight
    pbar = trange(time_steps, display=display)
    for idx in pbar:
        y = w_out(x)  # RIGHT Implement y[k] = w^out x[k].
        d = ds[idx] if (ds is not None) and (idx < len(ds)) else 0.0
        if idx % force_every == 0 and (train_range[0] <= idx < train_range[1]):
            dws = w_out.step(x, d)  # RIGHT Update weight.
            pbar.set_description("|ΔW|={:.3e}".format(np.linalg.norm(dws)))
        x = net(x, w_feed(y))  # RIGHT Implement x[k + 1] = f(x[k], y[k])
        record["x"][idx + 1] = x
        record["y"][idx] = y
        record["d"][idx] = d
        record["w"][idx + 1] = w_out.weight
    return record


def solution(time_steps, x0, net, w_feed, dim, dim_in, **kwargs):
    w_out = FORCEReadout(dim, dim_in)
    record = emulate_online(time_steps, x0, net, w_feed, w_out, **kwargs)
    return record["x"], record["y"]


test_func(solution, "03_02", multiple_output=True)
# show_solution("03_02", "emulate_online")  # Uncomment it to see the solution.

[ja]: #
#### 周期関数の埋め込み

[en]: #
#### Periodic function

[ja]: #
まずは元論文<sup>[5]</sup>と同様に周期関数の埋め込みを試しましょう。
以下の式で表される目標時系列を用意します。

[en]: #
First, let's try embedding a periodic function as shown in the original paper<sup>[5]</sup>.
Use the following sinusoidal function.

[END]: #

$$
\begin{align*}\\
d(t) =
\frac{2}{3} \sin\left(\frac{\pi t}{T}\right) +
\frac{1}{3} \sin\left(\frac{2\pi t}{T}\right) +
\frac{1}{9} \sin\left(\frac{3\pi t}{T}\right) +
\frac{2}{9} \sin\left(\frac{4\pi t}{T}\right)
,\end{align*}
$$

[ja]: #
ただし $T=60$ です。

[en]: #
where the time constant $T=60$.

In [ ]:
ts = np.linspace(0, 20000, 20001)

amps = [2 / 3, 1 / 3, 1 / 9, 2 / 9]
freqs = [1.0, 2.0, 3.0, 4.0]

ds_p = np.zeros_like(ts)
for a, f in zip(amps, freqs, strict=True):
    ds_p += a * np.sin(f * np.pi * ts / 60)
ds_p = ds_p / 1.5

fig = show_trajectory(ds_p[:1000])

[ja]: #
先ほど学習に失敗したネットワークと同じ重み行列で、漏れ率  $a=0.1$ を加えたリーキーESNを用意します。
出力されるグラフのうちピンク色の領域において、FORCE学習によって $W^\mathrm{out}$ が更新されています。

[en]: #
The following cell prepares a leaky ESN with the same weight matrix as the network that failed to train earlier but with a leaky rate $a=0.1$.
In the pink region of the output graph, $W^\mathrm{out}$ is updated by FORCE learning.

In [ ]:
seed = 1234
rnd = np.random.default_rng(seed)

net_dim = 1000
out_dim = 1

net = ESN(net_dim, sr=1.5, p=0.01, a=0.1, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=5.0, bias=0.2, rnd=rnd)
w_out = FORCEReadout(net_dim, out_dim, lmbd=0.1, rnd=rnd, initialize_with_zero=True)
x0 = rnd.uniform(-1, 1, net_dim)

t_total = 5000
train_range = [1000, 4000]
force_every = 2

record_t = emulate_online(
    t_total,
    x0,
    net,
    w_feed,
    w_out,
    ds=ds_p,
    force_every=force_every,
    train_range=train_range,
)
fig = show_record(record_t, ["x", "y", "dw"])

[ja]: #
目的の軌道をどれだけうまく出力されているかを時間遅れ座標で確認しましょう。

[en]: #
The following cell demonstrates how successful the embedding is in time-delay coordinates.

In [ ]:
x1 = np.array(record_t["x"][-1])
record_e = emulate_online(1000, x1, net, w_feed, w_out, ds=ds_p[t_total:])
fig = show_delayed_coord(record_e["y"][:, 0], record_e["d"][:, 0], tau=10, labels=["output", "desired"])

[ja]: #
#### ローレンツ系の埋め込み

[en]: #
#### Lorenz system

[ja]: #
同様に、同じパラメータのネットワークでローレンツ系の埋め込みを試しましょう。

[en]: #
Similarly, let's try embedding the Lorenz system using a network with the same parameters.

In [ ]:
seed = 1234
rnd = np.random.default_rng(seed)

net_dim = 1000
out_dim = 3

net = ESN(net_dim, sr=1.5, p=0.01, a=0.1, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=5.0, bias=0.2, rnd=rnd)
w_out = FORCEReadout(net_dim, out_dim, lmbd=0.1, rnd=rnd, initialize_with_zero=True)
x0 = rnd.uniform(-1, 1, net_dim)

t_total = 5000
train_range = [1000, 4000]
force_every = 2

ds = ds_lz[:, :out_dim] * 0.01
record_t = emulate_online(
    t_total,
    x0,
    net,
    w_feed,
    w_out,
    ds=ds,
    force_every=force_every,
    train_range=train_range,
)
fig = show_record(record_t, ["x", "y", "dw"])

x1 = np.array(record_t["x"][-1])
record_e = emulate_online(10000, x1, net, w_feed, w_out, ds=ds[t_total:])

[ja]: #
同じく3次元座標上でアトラクタの形を確認しましょう。

[en]: #
Let's also check the shape of the attractor in 3D coordinates.

In [ ]:
fig = show_3d_coord(output=record_e["y"], desired=record_e["d"])
fig.show()

[ja]: #
今度は同じESNパラメータでローレンツアトラクターの一部だけを提示してうまく表示できるか試しましょう（`out_dim=1`）。
時間遅れにより3次元表示を行います。

[en]: #
Now let's embed the Lorenz attractor based on partial observation (`out_dim=1`).
A 3D plot will be shown in the following cell.

In [ ]:
seed = 1234
rnd = np.random.default_rng(seed)

net_dim = 1000
out_dim = 1  # NOTE 3 to 1

net = ESN(net_dim, sr=1.5, p=0.01, a=0.1, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=10.0, bias=0.5, rnd=rnd)
w_out = FORCEReadout(net_dim, out_dim, lmbd=0.1, rnd=rnd, initialize_with_zero=True)
x0 = rnd.uniform(-1, 1, net_dim)

t_total = 7000
train_range = [1000, 6000]
force_every = 2

ds = ds_lz[:, :out_dim] * 0.01
record_t = emulate_online(
    t_total,
    x0,
    net,
    w_feed,
    w_out,
    ds=ds,
    force_every=force_every,
    train_range=train_range,
)
fig = show_record(record_t, ["x", "y", "dw"])

x1 = np.array(record_t["x"][-1])
record_e = emulate_online(10000, x1, net, w_feed, w_out, ds=ds[t_total:])

[ja]: #
遅延遅れ座標形状でのアトラクタの形を確認しましょう。

[en]: #
Let's check the shape of the attractor in time-delay coordinates.

In [ ]:
data1 = construct_delayed_coord(record_e["y"], 10, 3)
data2 = construct_delayed_coord(record_e["d"], 10, 3)
fig = show_3d_coord(output=data1, desired=data2)
fig.show()

[ja]: #
#### 複数のアトラクタの埋め込み

[en]: #
#### Embedding multiple attractors

[ja]: #
同じESNに複数のアトラクターを埋め込めます。
以下の設問では複数のアトラクタに対して学習できるように `emulate_online` を拡張した `emulate_parallel` を実装しましょう。

[en]: #
You can embed multiple attractors within the same system.
Implement `emulate_parallel`, an extension of `emulate_online` that can learn multiple attractors at the same time:

[END]: #
$$
\begin{align*}
{x}[k+1] &= (1-a){x}[k]+a\tanh\left(\rho W^\text{rec} {x}[k] + W^\mathrm{feed}{y}[k] + W^\mathrm{in} u[k]\right)\\
{y}[k] &= W^{\text{out}} {x}[k]
,\end{align*}
$$

[ja]: #
ここで、$u$ は制御パラメータ、$W^\mathrm{in} \in \mathbb{R}^{N}$ は $u$ をESNに投影する行列です。
それぞれ $u=-1$ と $u=1$ に設定したときに、同じ線形重みで、ローレンツ アトラクターと [L. Choingxinら<sup>[6]</sup>](https://www.sciencedirect.com/science/article/pii/S0960077904005909) によって提案された別の蝶型アトラクターを出力する閉ループ系を設計してみましょう。

[en]: #
where $u$ is a control parameter, and $W^\text{in} \in \mathbb{R}^{N}$ is a matrix that projects $u$ into the high-dimensional states of the ESN.
Let's design a feedback loop that outputs the Lorenz attractor and another butterfly-shaped attractor proposed by [L. Chongxin *et al.*<sup>[6]</sup>](https://www.sciencedirect.com/science/article/pii/S0960077904005909) when $u=-1$ and $u=1$ are given, respectively.

Q3.3.

[ja]: #
以下の空欄を埋め、FORCE学習によって **複数提示された** $d(t)$ に対して$W^\mathrm{out}$ を時々刻々更新する `emulate_parallel` を完成させよ。
ただし $k\in[t_0, t_1) \land k \equiv 0~ (\mathrm{mod}~t_\mathrm{every}) $ のときに $W^\mathrm{out}$ を更新、それ以外のときは変更はない。

[en]: #
Implement `emulate_parallel` using FORCE learning to continually update $W^\mathrm{out}$ for **multiple** $d(t)$.
$W^\mathrm{out}$ is updated when $k\in[t_0, t_1) \land k \equiv 0~ (\mathrm{mod}~t_\mathrm{every}) $, and is unchanged otherwise.

[END]: #
- `emulate_parallel`:
  - Argument(s):
    - `time_steps`: `int`
      - $T$
    - `x0`: `np.ndarray`
      - $x[0]$
    - `net`: `ESN`
    - `w_feed`: `Linear | RidgeReadout`
      - $W^\mathrm{feed}$
    - `w_out`: `Linear | RidgeReadout`
      - $W^\mathrm{out}$
    - `w_in`: `Linear | RidgeReadout`
      - $W^\mathrm{in}$
    - `ds`: `np.ndarray`
      - `shape`: `(t, k, dim_out)`
      - $[d[0], d[1],~\ldots]$
    - `us`: `np.ndarray`
      - `shape`: `(t, k, dim_in)`
      - $[u[0], u[1],~\ldots]$
    - `force_every`: `int`
      - $t_\mathrm{every}$
    - `train__range`: `tuple(int, int)`
      - $[t_0, t_1)$
  - Return(s):
    - `record`: `dict`
      - `'t'`: `np.ndarray`
        - $[0, 1, \ldots, T-1, T]$
      - `'x'`: `np.ndarray`
        - $[x[0], x[1], \ldots, x[T-1], x[T]]$
      - `'y'`: `np.ndarray`
        - $[y[0], y[1], \ldots, y[T-1]]$
      - `'d'`: `np.ndarray`
        - $[d[0], d[1], \ldots, d[T-1]]$
      - `'w'`: `np.ndarray`
        - $[W^\mathrm{out}[0], W^\mathrm{out}[1], \ldots, W^\mathrm{out}[T-1], W^\mathrm{out}[T]]$

In [ ]:
def emulate_parallel(
    time_steps,
    x0,
    net,
    w_feed,
    w_in,
    w_out,
    ds=None,
    us=None,
    train_range=None,
    force_every=1,
    display=True,
):
    train_range = train_range if train_range is not None else [0, 0]
    record = {}
    record["t"] = np.arange(0, time_steps + 1)
    record["x"] = np.zeros((time_steps + 1, *x0.shape))
    record["y"] = np.zeros((time_steps, *w_out(x0).shape))
    record["d"] = np.zeros((time_steps, *w_out(x0).shape))
    record["w"] = np.zeros((time_steps + 1, *w_out.weight.shape))
    record["train_range"] = train_range

    x = x0
    record["x"][0] = x0
    record["w"][0] = w_out.weight
    pbar = trange(time_steps, display=display)
    for idx in pbar:
        y = w_out(x)  # RIGHT Implement y[k] = w^out x[k].
        d = ds[idx] if (ds is not None) and (idx < len(ds)) else 0.0
        u = us[idx] if (us is not None) and (idx < len(us)) else 0.0
        if idx % force_every == 0 and (train_range[0] <= idx < train_range[1]):
            if x.ndim == 1:
                dws = w_out.step(x, d)  # RIGHT Implement weight update for 1D case.
            elif x.ndim == 2:
                dws = []
                for idy in range(x.shape[0]):
                    out = w_out.step(x[idy], d[idy])  # RIGHT Implement weight update for 2D case.
                    dws.append(out)
            pbar.set_description("|ΔW|={:.3e}".format(np.linalg.norm(dws)))
        x = net(x, w_feed(y) + w_in(u))  # RIGHT Implement x[k + 1] = f(x[k], y[k], u[k])
        record["x"][idx + 1] = x
        record["y"][idx] = y
        record["d"][idx] = d
        record["w"][idx + 1] = w_out.weight
    return record


def solution(time_steps, x0, net, w_feed, w_in, dim, dim_in, **kwargs):
    w_out = FORCEReadout(dim, dim_in)
    record = emulate_parallel(time_steps, x0, net, w_feed, w_in, w_out, **kwargs)
    return record["x"], record["y"]


test_func(solution, "03_03", multiple_output=True)
# show_solution("03_03", "emulate_parallel")  # Uncomment it to see the solution.

[ja]: #
`chongxin_func` は以下の微分方程式を実装しています。

[en]: #
`chongxin_func` implements the following differential equation:

[END]: #
$$
\begin{align*}
\frac{d x_1}{d t} &= a(x_2 - x_1) \\
\frac{d x_2}{d t} &= b x_1 - k x_1 x_3 \\
\frac{d x_3}{d t} &= - c x_3 + h x_1^2
.\end{align*}
$$

[ja]: #
この微分方程式を時間発展させると、次のセルで表示されるようにローレンツ系の蝶形構造の中央に渦が追加されるような軌跡が描かれます。

[en]: #
When this differential equation is evolved over time, it produces a trajectory where a vortex is added to the center of the butterfly structure of the Lorenz system, as shown in the next cell.

In [ ]:
from utils.chaos import chongxin_func

# Uncomment it to see the implementations.
# chongxin_func??

dt = 0.01
time_steps = 20000
z0 = np.array([1.0, 1.0, 1.0])
chongxin_params = dict(a=10.0, b=28.0, c=8.0 / 3.0, k=1, h=4)
chongxin_func_rk4 = runge_kutta(dt, chongxin_func, **chongxin_params)

ds_cx = np.zeros((time_steps + 1, 3))
ds_cx[0] = z0
for idx in trange(time_steps):
    ds_cx[idx + 1] = chongxin_func_rk4(ds_cx[idx])

[ja]: #
計算が完了したら、3次元座標上でアトラクタの形を確認しましょう。

[en]: #
Once the computation is complete, let's check the shape of the attractor in three-dimensional coordinates.

In [ ]:
fig = show_3d_coord(lorenz=ds_lz[:5000], chongxin=ds_cx[:5000])
fig.show()

[ja]: #
ご覧のとおりこの2つのアトラクタは重複が多いです。
重複している点ではその履歴に応じて次の出力が決定されなければなりません。
以下のセルはこの新しいアトラクタと、ローレンツアトラクタを、同じリードアウト層 $W^\mathrm{out}$ で入力に応じて別々に出力するように学習します。
果たしてうまく行くでしょうか？

[en]: #
As you can see, these two attractors overlap significantly.
At overlapping points, the next output must be determined based on the history (the sequence of past states).
The following cell trains the system to output this new attractor and the Lorenz attractor separately depending on the input, using the same readout layer $W^\mathrm{out}$.
Will it work properly?

In [ ]:
seed = 1234
rnd = np.random.default_rng(seed)

net_dim = 1000
out_dim = 3

net = ESN(net_dim, sr=1.5, p=0.01, a=0.1, rnd=rnd)
w_feed = Linear(out_dim, net_dim, bound=5.0, bias=0.5, rnd=rnd)
w_in = Linear(1, net_dim, bound=0.1, rnd=rnd)
w_out = FORCEReadout(net_dim, out_dim, lmbd=0.1, rnd=rnd, initialize_with_zero=True)

x0 = rnd.uniform(-1, 1, (2, net_dim))

t_total = 10000
train_range = [1000, 6000]
force_every = 2


ds_all = np.array([ds_lz * 0.01, ds_cx * 0.01]).swapaxes(0, 1)
us_all = np.zeros((len(ds_all), 2, 1))
us_all[:, 1] = 1  # u=1 for lorenz
us_all[:, 0] = -1  # u=-1 for chongxin

record_t = emulate_parallel(
    t_total,
    x0,
    net,
    w_feed,
    w_in,
    w_out,
    ds=ds_all,
    us=us_all,
    force_every=force_every,
    train_range=train_range,
)

fig_lz = show_record(record_t, ["x", "y", "dw"], view_slice=(..., 0))
fig_lz[0].set_title("Lorenz system")
fig_cx = show_record(record_t, ["x", "y", "dw"], view_slice=(..., 1))
fig_cx[0].set_title("Chongxin system")

x1 = np.array(record_t["x"][-1])
record_e = emulate_parallel(10000, x1, net, w_feed, w_in, w_out, ds=ds_all[t_total:], us=us_all[t_total:])

[ja]: #
同じく、3次元座標上でアトラクタの形を確認しましょう。

[en]: #
Similarly, let's check the shape of the attractor in 3D coordinates.

In [ ]:
fig = show_3d_coord(
    lorenz_out=record_e["y"][:, 0],
    lorenz_des=record_e["d"][:, 0],
    chongxin_out=record_e["y"][:, 1],
    chongxin_des=record_e["d"][:, 1],
)
fig.show()

[ja]: #
`..._des` が目標軌道 `..._out` が実際の出力を表します。

[en]: #
`..._des` represents the desired trajectory and `..._out` represents the actual output.

[ja]: #
Q3.4. (Advanced)
- 上記の切り替えタスクで動的に $u[k]$ を変化させたとき何が生じるか確認せよ。
- $-1$ と $1$ の中間の値を与えたときのアトラクタを描画せよ。
- 他にも様々なアトラクタの埋め込みを試し、FORCE学習の利点と限界を探れ。

[en]: #
Q3.4. (Advanced)
- Check what happens when $u[k]$ is changed dynamically in the switching task above.
- Plot the attractor when $u$ is set to intermediate values between $-1$ and $1$.
- Try embedding various other attractors and explore the advantages and limitations of FORCE learning.

[ja]: #
## 参考文献

[en]: #
## References

[1] Horii, Y., Inoue, K., Nishikawa, S., Nakajima, K., Niiyama, R., & Kuniyoshi, Y. (2021, July 19). *Physical reservoir computing in a soft swimming robot*. ALIFE 2021: The 2021 Conference on Artificial Life. https://doi.org/10.1162/isal_a_00426

[2] DePasquale, B., Cueva, C. J., Rajan, K., Escola, G. S., & Abbott, L. F. (2018). *full-FORCE: A target-based method for training recurrent networks*. PLOS ONE, 13(2), e0191527. https://doi.org/10.1371/journal.pone.0191527

[3] Laje, R., & Buonomano, D. V. (2013). *Robust timing and motor patterns by taming chaos in recurrent neural networks*. Nature Neuroscience, 16(7), 925–933. https://doi.org/10.1038/nn.3405

[4] Jaeger, H., & Haas, H. (2004). *Harnessing Nonlinearity: Predicting Chaotic Systems and Saving Energy in Wireless Communication*. Science, 304(5667), 78–80. https://doi.org/10.1126/science.1091277

[5] Sussillo, D., & Abbott, L. F. (2009). *Generating Coherent Patterns of Activity from Chaotic Neural Networks*. Neuron, 63(4), 544–557. https://doi.org/10.1016/j.neuron.2009.07.018

[6] Chongxin, L., Ling, L., Tao, L., & Peng, L. (2006). *A new butterfly-shaped attractor of Lorenz-like system*. Chaos, Solitons & Fractals, 28(5), 1196–1203. https://doi.org/10.1016/j.chaos.2004.09.111